# SENTIMENTAL ANALYSIS - SPAM DETECTION

# 1. Install and Import Dependencies

In [11]:
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


ERROR: Could not find a version that satisfies the requirement torch==1.8.1+cu111 (from versions: 2.0.0, 2.0.0+cpu, 2.0.0+cu117, 2.0.0+cu118, 2.0.1, 2.0.1+cpu, 2.0.1+cu117, 2.0.1+cu118, 2.1.0, 2.1.0+cpu, 2.1.0+cu118, 2.1.0+cu121, 2.1.1, 2.1.1+cpu, 2.1.1+cu118, 2.1.1+cu121, 2.1.2, 2.1.2+cpu, 2.1.2+cu118, 2.1.2+cu121, 2.2.0, 2.2.0+cpu, 2.2.0+cu118, 2.2.0+cu121, 2.2.1, 2.2.1+cpu, 2.2.1+cu118, 2.2.1+cu121, 2.2.2, 2.2.2+cpu, 2.2.2+cu118, 2.2.2+cu121, 2.3.0, 2.3.0+cpu, 2.3.0+cu118, 2.3.0+cu121, 2.3.1, 2.3.1+cpu, 2.3.1+cu118, 2.3.1+cu121, 2.4.0, 2.4.1, 2.5.0)
ERROR: No matching distribution found for torch==1.8.1+cu111


In [12]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [47]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

# 2. Instantiate Model

In [48]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# 3. Encode and Calculate Sentiment

In [49]:
tokens = tokenizer.encode('URGENT: Your account has been compromised. Verify your identity immediately.', return_tensors='pt')

In [50]:
result = model(tokens)

In [51]:
result.logits

tensor([[ 0.7315,  0.0907,  0.0408, -0.3277, -0.3396]],
       grad_fn=<AddmmBackward0>)

In [52]:
int(torch.argmax(result.logits))+1

1

# 4. Load Dataframe

In [53]:
import numpy as np
import pandas as pd

In [54]:
df = pd.read_csv("D:/2024/NCI/Semester 3/Practicum 2/GitHub/LLM projects/Dataset/spam.csv", encoding='ISO-8859-1')

#df = pd.DataFrame(np.array(reviews), columns=['review'])
df["v2"]

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                Will Ì_ b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: v2, Length: 5572, dtype: object

In [55]:
df['v2'].iloc[0]

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [56]:
def sentiment_score(spam):
    tokens = tokenizer.encode(spam, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [57]:
sentiment_score(df['v2'].iloc[1])

3

In [58]:
# 512 take only the first 512 characters
df['sentiment'] = df['v2'].apply(lambda x: sentiment_score(x[:512]))

In [59]:
df

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4,sentiment
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN,1
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN,3
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN,4
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN,1
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN,3
...,...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN,1
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN,1
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN,2
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN,3


In [60]:
df.to_csv('sentiment.csv')

In [38]:
df['v2'].iloc[5]

"FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv"

In [40]:
#df_filtered_5 = df[df['sentiment'] == 5][['v1', 'v2', 'sentiment']]
#df_filtered_5


,v1,v2,sentiment
8,spam,WINNER!! As a valued network customer you have...,5
11,spam,"SIX chances to win CASH! From 100 to 20,000 po...",5
12,spam,URGENT! You have won a 1 week FREE membership ...,5
13,ham,I've been searching for the right words to tha...,5
14,ham,I HAVE A DATE ON SUNDAY WITH WILL!!,5
...,...,...,...
5550,ham,"Cool, what time you think you can get here?",5
5551,ham,Wen did you get so spiritual and deep. That's ...,5
5552,ham,Have a safe trip to Nigeria. Wish you happines...,5
5553,ham,Hahaha..use your brain dear,5


# nuevo SVM

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load data
df = pd.read_csv("sentiment.csv")  

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Extract BERT embeddings
def sentiment_score(spam):
    tokens = tokenizer.encode(spam, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1



# Apply BERT embedding extraction to all rows
df['bert_embedding'] = df['v2'].apply(get_bert_embedding)

# Split the dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(
    list(df['bert_embedding']),
    df['sentiment'],
    test_size=0.2,
    random_state=42
)

# Train an SVM model on the BERT embeddings
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

# Predict and evaluate
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


# BERT 1ST

In [6]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel

# Load the data
df = pd.read_csv('D:/2024/NCI/Semester 3/Practicum 2/GitHub/BERT test/BERTSentiment/spam.csv',encoding = 'latin-1')


# Check the data
print(df.head())

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to generate embeddings
def get_bert_embeddings(text):
    # Tokenize and encode the text
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    
    # Get the BERT output
    with torch.no_grad():
        outputs = model(**inputs)

    # Take the embedding of the [CLS] token
    cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze()
    return cls_embedding

# Generate embeddings for each text in the dataframe
df['embeddings'] = df['v2'].apply(get_bert_embeddings)

# Example to see the embeddings
print(df[['v2', 'embeddings']].head())


     v1                                                 v2 Unnamed: 2  \
0   ham  Go until jurong point, crazy.. Available only ...        NaN   
1   ham                      Ok lar... Joking wif u oni...        NaN   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   ham  U dun say so early hor... U c already then say...        NaN   
4   ham  Nah I don't think he goes to usf, he lives aro...        NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN  


AttributeError: module 'torch' has no attribute 'frombuffer'

In [4]:
pip install --upgrade torch



  Using cached https://files.pythonhosted.org/packages/9d/58/37166e7f71cd35e7f76d462fa058758a0aa6c31c1a7e26290f7898d37360/torch-2.4.1-cp38-cp38-win_amd64.whl


ERROR: torchvision 0.9.1+cu111 has requirement torch==1.8.1, but you'll have torch 2.4.1 which is incompatible.
ERROR: torchaudio 0.8.1 has requirement torch==1.8.1, but you'll have torch 2.4.1 which is incompatible.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'C:\\Python3.8\\Lib\\site-packages\\~orch\\lib\\asmjit.dll'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


  Using cached https://files.pythonhosted.org/packages/99/ff/c87e0622b1dadea79d2fb0b25ade9ed98954c9033722eb707053d310d4f3/sympy-1.13.3-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/43/e3/7d92a15f894aa0c9c4b49b8ee9ac9850d6e63b03c9c32c0367a13ae62209/mpmath-1.3.0-py3-none-any.whl
  Found existing installation: torch 1.8.1+cu111
    Uninstalling torch-1.8.1+cu111:
      Successfully uninstalled torch-1.8.1+cu111


In [5]:
pip install --upgrade transformers safetensors


  Using cached https://files.pythonhosted.org/packages/db/88/1ef8a624a33d7fe460a686b9e0194a7916320fc0d67d4e38e570beeac039/transformers-4.46.0-py3-none-any.whl
Requirement already up-to-date: safetensors in c:\python3.8\lib\site-packages (0.4.5)
Note: you may need to restart the kernel to use updated packages.Requirement already satisfied, skipping upgrade: pyyaml>=5.1 in c:\python3.8\lib\site-packages (from transformers) (5.4.1)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.



  Found existing installation: transformers 4.45.2
    Uninstalling transformers-4.45.2:
      Successfully uninstalled transformers-4.45.2


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [1]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel

# Load the data
df = pd.read_csv('D:/2024/NCI/Semester 3/Practicum 2/GitHub/BERT test/BERTSentiment/spam.csv',encoding = 'latin-1')


# Check the data
print(df.head())

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to generate embeddings
def get_bert_embeddings(text):
    # Tokenize and encode the text
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    
    # Get the BERT output
    with torch.no_grad():
        outputs = model(**inputs)

    # Take the embedding of the [CLS] token
    cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze()
    return cls_embedding

# Generate embeddings for each text in the dataframe
df['embeddings'] = df['v2'].apply(get_bert_embeddings)

# Example to see the embeddings
print(df[['v2', 'embeddings']].head())


c:\Python3.8\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


     v1                                                 v2 Unnamed: 2  \
0   ham  Go until jurong point, crazy.. Available only ...        NaN   
1   ham                      Ok lar... Joking wif u oni...        NaN   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   ham  U dun say so early hor... U c already then say...        NaN   
4   ham  Nah I don't think he goes to usf, he lives aro...        NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN  
                                                  v2  \
0  Go until jurong point, crazy.. Available only ...   
1                      Ok lar... Joking wif u oni...   
2  Free entry in 2 a wkly comp to win FA Cup fina...   
3  U dun say so early hor... U c already then say...   
4  Nah I don't think he goes to usf, he lives aro...   

                                          embeddings  
0  [tensor(-0

In [2]:
df[['v2', 'embeddings']]

,v2,embeddings
0,"Go until jurong point, crazy.. Available only ...","[tensor(-0.1513), tensor(-0.3229), tensor(0.18..."
1,Ok lar... Joking wif u oni...,"[tensor(-0.1238), tensor(0.3435), tensor(-0.00..."
2,Free entry in 2 a wkly comp to win FA Cup fina...,"[tensor(-0.5031), tensor(-0.2829), tensor(0.57..."
3,U dun say so early hor... U c already then say...,"[tensor(0.0646), tensor(0.5769), tensor(0.3411..."
4,"Nah I don't think he goes to usf, he lives aro...","[tensor(0.0947), tensor(0.3897), tensor(0.0389..."
...,...,...
5567,This is the 2nd time we have tried 2 contact u...,"[tensor(-0.0725), tensor(0.0551), tensor(0.620..."
5568,Will Ì_ b going to esplanade fr home?,"[tensor(-0.0089), tensor(0.1375), tensor(0.270..."
5569,"Pity, * was in mood for that. So...any other s...","[tensor(-0.0464), tensor(0.0716), tensor(-0.28..."
5570,The guy did some bitching but I acted like i'd...,"[tensor(0.2887), tensor(0.2153), tensor(-0.353..."


In [3]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np

# Convert labels to numerical values (e.g., "ham" -> 0, "spam" -> 1)
df['label'] = df['v1'].apply(lambda x: 1 if x == 'spam' else 0)

# Convert embeddings to a 2D array format
X = np.array([embedding.numpy() for embedding in df['embeddings']])
y = df['label'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the SVM classifier
svm_model = svm.SVC(kernel='linear')
svm_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = svm_model.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.99      0.99      0.99       965
           1       0.95      0.96      0.96       150

    accuracy                           0.99      1115
   macro avg       0.97      0.98      0.98      1115
weighted avg       0.99      0.99      0.99      1115

